# Example of DOV search methods for hydrogeological interpretations

@pjhaest
date 20180515

## Use cases:
* Select interpretations in a bbox
* Select interpretations in a bbox with selected properties
* Select interpretations in a municipality
* Get interpretations using info from wfs fields, not available in the standard output dataframe

In [1]:
%matplotlib inline
import os, sys
import inspect
import warnings; warnings.simplefilter('ignore')

In [2]:
# check pydov path
import pydov
print(pydov.__path__)

['c:\\projecten\\pydov\\pydov_git\\pydov']


## Get information about code base

In [3]:
from pydov.search.interpretaties import HydrogeologischeStratigrafieSearch
ip_hydrogeo = HydrogeologischeStratigrafieSearch()

# information about the HydrogeologischeStratigrafie type (In Dutch):
print(ip_hydrogeo.get_description())

De hydrostratigrafie geeft, op basis van de (gecodeerde) lithologie, een indeling weer naar de al dan niet watervoerende eigenschappen van een bepaald beschreven diepte-interval. Deze interpretatie respecteert de lithostratigrafie van het Tertiair, maar deelt deze anders in. De hiervoor gebruikte standaard is de Hydrogeologische Codering van de Ondergrond van Vlaanderen (HCOV). Deze kan beschouwd worden als de officiele hydrogeologische codering voor het Vlaams Gewest.


In [4]:
# information about the available fields for a HydrogeologischeStratigrafie object
fields = ip_hydrogeo.get_fields()
# print available fields
for f in fields.values():
    print(f['name'])

generated_id
Auteurs
Opdrachten
pkey_sondering
diepte_laag_van
diepte_tot_m
diepte_laag_tot
aquifer
Datum
Proeffiche
Z_mTAW
Geldig_van
Proefnummer
pkey_boring
gemeente
betrouwbaarheid_interpretatie
x
y
pkey_interpretatie
Type_proef
Geldig_tot


In [5]:
# print information for a certain field
fields['aquifer']

{'cost': 10,
 'definition': 'code van de watervoerende laag waarin de laag Hydrogeologische stratigrafie zich bevindt.',
 'name': 'aquifer',
 'notnull': False,
 'type': 'string'}

The cost is an arbitrary attribute to indicate if the information is retrieved from a wfs query (cost = 1), 
or from an xml (cost = 10)

In [6]:
# if an attribute can have several values, these are listed under 'values', e.g. for 'Type_proef':
fields['Type_proef']

{'cost': 1,
 'definition': "Het type van de proef waarvan de hydrogeologische stratigrafie een interpretatie is. Bij een hydrogeologische stratigrafie zal dit altijd 'Boring' zijn.",
 'name': 'Type_proef',
 'notnull': False,
 'type': 'string',
 'values': ['Boring']}

## Try-out of use cases

### Select interpretations in a bbox

In [7]:
from pydov.util.location import Within, Box

# Get all borehole data in a bounding box (llx, lly, ulx, uly)
# the pkey_boring link is not available below, but is in the df
df = ip_hydrogeo.search(location=Within(Box(153145, 206930, 153150, 206935)))
print(df.head())

[000/001] .
                                  pkey_interpretatie  \
0  https://www.dov.vlaanderen.be/data/interpretat...   
1  https://www.dov.vlaanderen.be/data/interpretat...   

                                         pkey_boring  pkey_sondering  \
0  https://www.dov.vlaanderen.be/data/boring/1974...             NaN   
1  https://www.dov.vlaanderen.be/data/boring/1974...             NaN   

  betrouwbaarheid_interpretatie         x         y  diepte_laag_van  \
0                          goed  153147.0  206931.0              0.0   
1                          goed  153147.0  206931.0              2.5   

   diepte_laag_tot aquifer  
0              2.5    0152  
1              7.0    0254  


### Select interpretations in a bbox with selected properties

In [8]:
# list available query methods
methods = [i for i,j in inspect.getmembers(sys.modules['owslib.fes'], 
                                           inspect.isclass) 
           if 'Property' in i]
print(methods)
from owslib.fes import PropertyIsGreaterThanOrEqualTo

['PropertyIsBetween', 'PropertyIsEqualTo', 'PropertyIsGreaterThan', 'PropertyIsGreaterThanOrEqualTo', 'PropertyIsLessThan', 'PropertyIsLessThanOrEqualTo', 'PropertyIsLike', 'PropertyIsNotEqualTo', 'PropertyIsNull', 'SortProperty']


The property feature methodes listed above are available from the owslib module. These were not adapted for use in pydov.

In [9]:
# Get deep boreholes in a bounding box 
from owslib.fes import PropertyIsEqualTo
# the propertyname can be any of the fields of the hydrogeological interpretations object that belong to the wfs source
# the literal is always a string, no matter what its definition is in the boring object (string, float...)
query = PropertyIsGreaterThanOrEqualTo(
        propertyname='betrouwbaarheid_interpretatie', literal='goed')
df = ip_hydrogeo.search(location=Within(Box(153145, 206930, 153150, 206935)),
                        query=query
                       )
print(df.head())

[000/001] c
                                  pkey_interpretatie  \
0  https://www.dov.vlaanderen.be/data/interpretat...   
1  https://www.dov.vlaanderen.be/data/interpretat...   

                                         pkey_boring  pkey_sondering  \
0  https://www.dov.vlaanderen.be/data/boring/1974...             NaN   
1  https://www.dov.vlaanderen.be/data/boring/1974...             NaN   

  betrouwbaarheid_interpretatie         x         y  diepte_laag_van  \
0                          goed  153147.0  206931.0              0.0   
1                          goed  153147.0  206931.0              2.5   

   diepte_laag_tot aquifer  
0              2.5    0152  
1              7.0    0254  


### Select interpretations in a municipality

In [10]:
query = PropertyIsEqualTo(propertyname='gemeente',
                          literal='Aartselaar')
df = ip_hydrogeo.search(query=query)
print(df.head())

[000/006] ......
                                  pkey_interpretatie  \
0  https://www.dov.vlaanderen.be/data/interpretat...   
1  https://www.dov.vlaanderen.be/data/interpretat...   
2  https://www.dov.vlaanderen.be/data/interpretat...   
3  https://www.dov.vlaanderen.be/data/interpretat...   
4  https://www.dov.vlaanderen.be/data/interpretat...   

                                         pkey_boring  pkey_sondering  \
0  https://www.dov.vlaanderen.be/data/boring/1950...             NaN   
1  https://www.dov.vlaanderen.be/data/boring/1950...             NaN   
2  https://www.dov.vlaanderen.be/data/boring/1894...             NaN   
3  https://www.dov.vlaanderen.be/data/boring/1894...             NaN   
4  https://www.dov.vlaanderen.be/data/boring/1895...             NaN   

  betrouwbaarheid_interpretatie         x         y  diepte_laag_van  \
0                          goed  151457.0  201527.0              0.0   
1                          goed  151457.0  201527.0             49.0 

### Get interpretations using info from wfs fields, not available in the standard output dataframe

To keep the dataframe size acceptable, not all wfs fields are included in the standard output. However, one can use this information to select interpretations as illustrated below.
The available wfs fields can be viewed in the object catalog of the 'Hydrogeologische stratigrafie' wfs service, on-line [2018 05 16]: https://www.dov.vlaanderen.be/geonetwork/apps/tabsearch/index.html?hl=dut.
For example, make a selection of the interpretations in the municipality of Antwerp that were derived from cpt data

In [11]:
from owslib.fes import And
from owslib.fes import PropertyIsEqualTo
query = And([PropertyIsEqualTo(propertyname='gemeente',
                               literal='Antwerpen'),
             PropertyIsEqualTo(propertyname='Type_proef', 
                               literal='Boring')]
            )
df = ip_hydrogeo.search(query=query, )
print(df.head())

[000/195] ..................................................
[050/195] ..................................................
[100/195] ..................................................
[150/195] ..............c..............................
                                  pkey_interpretatie  \
0  https://www.dov.vlaanderen.be/data/interpretat...   
1  https://www.dov.vlaanderen.be/data/interpretat...   
2  https://www.dov.vlaanderen.be/data/interpretat...   
3  https://www.dov.vlaanderen.be/data/interpretat...   
4  https://www.dov.vlaanderen.be/data/interpretat...   

                                         pkey_boring  pkey_sondering  \
0  https://www.dov.vlaanderen.be/data/boring/1975...             NaN   
1  https://www.dov.vlaanderen.be/data/boring/1975...             NaN   
2  https://www.dov.vlaanderen.be/data/boring/1975...             NaN   
3  https://www.dov.vlaanderen.be/data/boring/1994...             NaN   
4  https://www.dov.vlaanderen.be/data/boring/1994...            

### Visualize results

Using Folium, we can display the results of our search on a map.

In [12]:
# import the necessary modules (not included in the requirements of pydov!)
import folium
from folium.plugins import MarkerCluster
from pyproj import Proj, transform

In [13]:
# convert the coordinates to lat/lon for folium
def convert_latlon(x1, y1):
    inProj = Proj(init='epsg:31370')
    outProj = Proj(init='epsg:4326')
    x2,y2 = transform(inProj, outProj, x1, y1)
    return x2, y2
df['lon'], df['lat'] = zip(*map(convert_latlon, df['x'], df['y'])) 
# convert to list
loclist = df[['lat', 'lon']].values.tolist()

In [14]:
# initialize the Folium map on the centre of the selected locations, play with the zoom until ok# initia 
fmap = folium.Map(location=[df['lat'].mean(), df['lon'].mean()], zoom_start=12)
marker_cluster = MarkerCluster().add_to(fmap)
for loc in range(0, len(loclist)):
    folium.Marker(loclist[loc], popup=df['aquifer'][loc]).add_to(marker_cluster)
fmap